## Interactividad (ejemplo simple)

En esta sección se puede enlazar interactividad con `ipywidgets` (selector de segmento, slider de tiempo, etc.). El ejemplo principal despliega el mapa y la gráfica; para integraciones más avanzadas puedo añadir controles que actualicen la gráfica al seleccionar un segmento en el mapa.

In [7]:
# Celda 7: Graficar perfil vertical con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=pos_df['chainage_m'], y=pos_df['disp_mm'], mode='lines+markers', name='Perfil vertical (mm)',
                         marker=dict(size=6, color='blue')))

# Añadir regiones semáforo como bandas de color (según calidad por segmento)
for _, s in seg_df.iterrows():
    x0 = pos_df['chainage_m'].iloc[s['start']]
    x1 = pos_df['chainage_m'].iloc[s['end']]
    color = 'rgba(0,255,0,0.08)' if s['quality']=='buena' else ('rgba(255,165,0,0.08)' if s['quality']=='regular' else 'rgba(255,0,0,0.08)')
    fig.add_vrect(x0=x0, x1=x1, fillcolor=color, layer='below', line_width=0)

fig.update_layout(title='Perfil vertical estimado (mm) vs Chainage (m)', xaxis_title='Chainage (m)', yaxis_title='Desplazamiento vertical (mm)', height=450)
fig.show()

KeyError: 'disp_mm'

In [6]:
# Celda 6: Procesar registros de aceleración para obtener perfil vertical (simplificado)
# Para cada punto integraremos la señal para obtener desplazamiento aproximado (doble integración con detrend)
from scipy.integrate import cumtrapz

displacements = []
for a in accel_data:
    a_detr = signal.detrend(a)
    # filtro pasa-bajo para reducir ruido de alta frecuencia antes de integrar
    b, c = signal.butter(3, 10.0/(fs/2), btype='low')
    a_f = signal.filtfilt(b, c, a_detr)
    v = cumtrapz(a_f, dx=1.0/fs, initial=0)
    d = cumtrapz(v, dx=1.0/fs, initial=0)
    # corremos corrección lineal simple para eliminar deriva
    p = np.polyfit(np.arange(len(d)), d, 1)
    d_corr = d - (p[0]*np.arange(len(d)) + p[1])
    # tomamos valor medio o pico-valor para representar perfil en ese punto
    displacements.append(d_corr.mean())

pos_df['disp_mm'] = np.array(displacements)*1000.0  # convertir a mm
pos_df.head()

ImportError: cannot import name 'cumtrapz' from 'scipy.integrate' (C:\Users\uli_r\anaconda3\Lib\site-packages\scipy\integrate\__init__.py)

In [5]:
# Celda 5: Crear mapa interactivo con Folium
m = folium.Map(location=[pos_df['lat'].mean(), pos_df['lon'].mean()], zoom_start=14, tiles='cartodbpositron')

# Añadir segmentos coloreados
for _, row in seg_df.iterrows():
    folium.PolyLine(locations=[(lat, lon) for lat,lon in row['coords']], color=row['color'], weight=6, opacity=0.8,
                    tooltip=f"Calidad: {row['quality']} — RMS={row['rms']:.5f}").add_to(m)

# Añadir marcadores de inicio/fin
folium.Marker(location=[lats[0], lons[0]], popup='Inicio', icon=folium.Icon(color='blue', icon='train')).add_to(m)
folium.Marker(location=[lats[-1], lons[-1]], popup='Fin', icon=folium.Icon(color='blue', icon='flag')).add_to(m)

# Leyenda personalizada
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 10px; width:150px; height:110px; background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
&nbsp;<b>Leyenda calidad</b><br>
&nbsp;<i style="background:green;color:green">....</i>&nbsp;Buena<br>
&nbsp;<i style="background:orange;color:orange">....</i>&nbsp;Regular<br>
&nbsp;<i style="background:red;color:red">....</i>&nbsp;Mala<br>
</div>
'''

m.get_root().html.add_child(branca.element.Element(legend_html))

# Guardar mapa temporalmente y mostrar iframe
mapfile = 'demo_map.html'
m.save(mapfile)

display(IFrame(mapfile, width='100%', height=450))


In [4]:
# Celda 4: Asignar calidad por segmento según RMS (semáforo)
# Definimos umbrales sencillos: RMS < 0.006 -> buena, <0.012 -> regular, >=0.012 -> mala
seg_qualities = []
for seg in segments:
    idx0 = seg['start_idx']
    idx1 = seg['end_idx']
    seg_rms = pos_df['rms_acc'].iloc[idx0:idx1+1].mean()
    if seg_rms < 0.006:
        q = 'buena'
        color = 'green'
    elif seg_rms < 0.012:
        q = 'regular'
        color = 'orange'
    else:
        q = 'mala'
        color = 'red'
    seg_qualities.append({'start': idx0, 'end': idx1, 'rms': float(seg_rms), 'quality': q, 'color': color, 'coords': seg['coords']})

seg_df = pd.DataFrame(seg_qualities)
seg_df.head()

,start,end,rms,quality,color,coords
0,0,5,0.008176,regular,orange,"[(40.0, -3.6990000000000003), (40.000241771687..."
1,5,10,0.014677,mala,red,"[(40.00113192630772, -3.698716014899819), (40...."
2,10,15,0.018814,mala,red,"[(40.00184024901613, -3.698667304132702), (40...."
3,15,20,0.019337,mala,red,"[(40.001953472614225, -3.69875269538413), (40...."
4,20,25,0.016155,mala,red,"[(40.001654275397144, -3.6988344915180393), (4..."


In [3]:
# Celda 3: Generar datos sintéticos de la vía (LineString) y segmentos
# Generamos una vía como una curva de ~100 puntos
n_points = 120
lats = 40.0 + 0.01 * np.linspace(0, 1, n_points) + 0.001 * np.sin(np.linspace(0, 6*np.pi, n_points))
lons = -3.7 + 0.01 * np.linspace(0, 1, n_points) + 0.001 * np.cos(np.linspace(0, 4*np.pi, n_points))
coords = list(zip(lats, lons))

# Dividir en segmentos (cada segmento = 5 puntos)
seg_size = 5
segments = []
for i in range(0, n_points-1, seg_size):
    seg_coords = coords[i:i+seg_size+1]
    segments.append({
        'start_idx': i,
        'end_idx': min(i+seg_size, n_points-1),
        'coords': seg_coords
    })

# Simular registros de aceleración por punto (tiempo breve por punto)
# Para cada punto generamos una serie temporal de aceleración vertical (m/s^2)
samples_per_point = 200
fs = 100.0  # Hz
accel_data = []
for i in range(n_points):
    t = np.arange(0, samples_per_point) / fs
    # combinación de sinusoides + ruido; amplitud variable por posición
    amp = 0.005 + 0.02 * np.abs(np.sin(i / 10.0))
    acc = amp * (np.sin(2*np.pi*1.5*t) + 0.5*np.sin(2*np.pi*6*t)) + 0.002*np.random.randn(samples_per_point)
    accel_data.append(acc)

# DataFrame con posiciones y una métrica simple: RMS de aceleración
rms = np.array([np.sqrt(np.mean(a**2)) for a in accel_data])
pos_df = pd.DataFrame({'lat': lats, 'lon': lons, 'rms_acc': rms})
pos_df['chainage_m'] = np.linspace(0, 1000, n_points)  # distancia simulada en metros
pos_df.head()

,lat,lon,rms_acc,chainage_m
0,40.000000,-3.699000,0.004335,0.000000
1,40.000242,-3.698922,0.005826,8.403361
2,40.000480,-3.698854,0.007603,16.806723
3,40.000710,-3.698798,0.008978,25.210084
4,40.000928,-3.698752,0.010491,33.613445


In [2]:
# Celda 2: Importar librerías
import numpy as np
import pandas as pd
import folium
import branca
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal
from IPython.display import IFrame, display

# Opciones para Jupyter
pd.options.display.max_columns = 50
np.random.seed(42)

# Dashboard demo: Mapa de calidad de vía (semáforo) y perfil vertical

Este notebook genera datos sintéticos de una vía férrea, asigna una calidad (buena/regular/mala) por segmento y muestra:
- Un mapa interactivo con la vía coloreada tipo semáforo (verde/amarillo/rojo) usando Folium.
- Una gráfica interactiva del perfil vertical obtenida desde registros de aceleración (procesamiento con SciPy y visualización con Plotly).

Ejecuta las celdas en orden. Instalar dependencias con `pip install -r requirements.txt` si es la primera vez.